<a href="https://colab.research.google.com/github/Jerry086/CS6140CarbonMapping/blob/main/Upadated%20Feature%20Stack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Forest Carbon Mapping

## Setting Up Enviornment

Mount Google Drive to the Colab VM

In [ ]:
# mount google drive if dataset is saved in it
# skip the code if dataset is uploaded or downloaded from google cloud
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Authenticate google cloud account

In [1]:
# authenticate google cloud if dataset is saved in it
# TODO: deploy T4 GPU on google cloud
from google.colab import auth
auth.authenticate_user()

Authenticate and authorize access to Earth Engine.

In [2]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=DXlbkt896kHZWOwY-DW8NqXuXNGMu_v092jkYkHsOGQ&tc=7vJ_rv_F3ocdVr-m5qYL6r4O3hRXZ58XwoB8I0PafB8&cc=vein0kWlvpD5hZ8dHPvncBQHcL792QL8pOd-BRCrvGg

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BXv9mKmFA5kyG_kKc7f8nWQ_lc7vWykeoacj7fc2wkdKHdQXWUchNc

Successfully saved authorization token.


## Install & import Python packages



Import Python packages used throughout this notebook.

In [3]:
# Folium setup.
import folium
print(folium.__version__)

0.14.0


In [4]:
# Tensorflow setup.
import tensorflow as tf
print(tf.__version__)

2.12.0


## Set global variables

In [5]:
# Specify names locations for outputs in Google Drive.
# Uncomment the code for Google Drive storage.
# FOLDER = 'carbon_mapping'

# Specify names locations for outputs in Google Cloud.
BUCKET = 'cs6140'
FOLDER = 'copernicus'
TRAINING_BASE = 'training_patches'
EVAL_BASE = 'eval_patches'

# Specify feature bands to the model and the response variable.
# TODO: explore more bands
MODIS_BANDS = ['NDVI', 'EVI']
COPERNICUS_BANDS = ['discrete_classification', 'discrete_classification-proba', 'forest_type']
TERRA_BANDS=['Percent_Tree_Cover', 'Percent_NonTree_Vegetation']
BANDS = MODIS_BANDS + COPERNICUS_BANDS + TERRA_BANDS
RESPONSE = 'annualNPP'
FEATURES = BANDS + [RESPONSE]

# Specify the size and shape of patches (256x256 pixels images) expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]

# Columns for input features and response
COLUMNS = [
  # Configuration for parsing a fixed-length input feature.
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
# Label each column with feature name by dictionary
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Sizes of the training and evaluation datasets.
# TODO: modify as needed
TRAIN_SIZE = 16000
EVAL_SIZE = 8000

# Specify model training parameters.
# TODO: modify as needed
BATCH_SIZE = 16
EPOCHS = 10
BUFFER_SIZE = 2000
OPTIMIZER = 'Adam'
LOSS = 'MeanSquaredError'
METRICS = ['RootMeanSquaredError']

# Visualizing Images and Image Bands

This research focuses on Forest Carbon Mapping in North America area

In [6]:
# Define the region of interest (ROI)
north_america_polygon = ee.Geometry.Polygon([
    [-168, 65],  # Northwest corner (top-left)
    [-168, 10],  # Southwest corner (bottom-left)
    [-52, 10],   # Southeast corner (bottom-right)
    [-52, 65],   # Northeast corner (top-right)
])

In [7]:
def reduce_collection(filename):
  """The filter function.
  Filter an image collection to the the year of 2018, region of North America
  Reduce temporal data by mean
  Args:
    filename: dataset filename of the collection.
  Returns:
    A representative image.
  """
  collection = ee.ImageCollection(filename)
  filtered = collection.filterDate('2018-01-01', '2018-12-31')
  reduced = filtered.mean().clip(north_america_polygon)
  return reduced

NDVI and EVI bands from [MOD13Q1.061 Terra Vegetation Indices 16-Day Global 250m](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD13Q1#bands).

In [8]:
modis_image = reduce_collection("MODIS/061/MOD13Q1")

# Use folium to visualize the image
mapid = modis_image.getMapId({'bands': ['NDVI'], 'min': 0, 'max': 8000, 'palette': [
    'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
    '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
    '012e01', '011d01', '011301'
  ]})
# Center in Vancouver
map = folium.Map(location=[49.2827, -123.1207])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='NDVI',
  ).add_to(map)

mapid = modis_image.getMapId({'bands': ['EVI'], 'min': 0, 'max': 5000, 'palette': [
    'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
    '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
    '012e01', '011d01', '011301'
  ]})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='EVI',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

Discrete_classification, discrete_classification-proba and forest_type bands from [Copernicus Global Land Cover Layers: CGLS-LC100 Collection 3](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global#bands).

In [9]:
copernicus_image = reduce_collection("COPERNICUS/Landcover/100m/Proba-V-C3/Global")

# Use folium to visualize the image
mapid = copernicus_image.getMapId({'bands': ['discrete_classification', 'discrete_classification-proba'], 'min': 0, 'max': 200})
map = folium.Map(location=[49.2827, -123.1207])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='discrete_classification',
  ).add_to(map)

mapid = copernicus_image.getMapId({'bands': ['forest_type'], 'min': 0, 'max': 5, 'palette': [
    '282828', '666000', '009900', '70663e', 'a0dc00', '929900'
  ]})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='forest_type',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [10]:
terra_image = reduce_collection("MODIS/006/MOD44B")

# Use folium to visualize the image
mapid = terra_image.getMapId({'bands': ['Percent_Tree_Cover', 'Percent_NonTree_Vegetation'], 'min': 0, 'max': 100})
map = folium.Map(location=[49.2827, -123.1207])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='discrete_classification',
  ).add_to(map)

mapid = terra_image.getMapId({'bands': ['Percent_Tree_Cover'], 'min': 0, 'max': 100, 'palette': ['bbe029', '0a9501', '074b03']})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='forest_type',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

Response: the amount of carbon captured by plants in an ecosystem, after accounting for losses due to respiration, from [MODIS Net Primary Production CONUS](https://developers.google.com/earth-engine/datasets/catalog/UMT_NTSG_v2_MODIS_NPP#bands).

In [ ]:
# npp_image = reduce_collection('UMT/NTSG/v2/MODIS/NPP')
# npp_image = npp_filtered.mean().select('annualNPP')

# # normalize npp image to [0, 1]
# stats = npp_image.reduceRegion(
#     reducer=ee.Reducer.minMax(),
#     geometry=north_america_polygon,
#     scale=250,
#     bestEffort=True
# ).getInfo()
# print(stats)
# min, max = stats['annualNPP_min'], stats['annualNPP_max']
# npp_normalized = npp_image.unitScale(min, max)

{'annualNPP_max': 29278, 'annualNPP_min': 0}


In [ ]:
# # inspect some points for validation
# points = [
#     ee.Geometry.Point(-115.16, 35.49),
#     ee.Geometry.Point(-90.54, 38.7)
# ]

# # Function to extract pixel values at specified points
# def get_pixel_values(point):
#     value_dict = npp_normalized.reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=250,bestEffort=True)
#     return value_dict

# # Loop through the points and get the pixel values
# for point in points:
#     pixel_values = get_pixel_values(point)
#     print(pixel_values.getInfo())

{'annualNPP': 0.030910581323860917}
{'annualNPP': 0.44173099255413617}


In [11]:
npp_image = reduce_collection('UMT/NTSG/v2/MODIS/NPP')

mapid = npp_image.getMapId({'bands': ['annualNPP'], 'min': 0, 'max': 20000, 'palette': ['bbe029', '0a9501', '074b03']})
map = folium.Map(location=[38., -122.5])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='npp',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

Stack the 2D images (Terra Vegetation Indices and MODIS Net Primary Production) to create a single image from which samples can be taken. Convert the image into an array image in which each pixel stores 256x256 patches of pixels for each band. To export training patches, convert a multi-band image to an array image using neighborhoodToArray(), then sample the image at points.


In [13]:
featureStack = ee.Image.cat([
  modis_image.select(MODIS_BANDS),
  copernicus_image.select(COPERNICUS_BANDS),
  terra_image.select(TERRA_BANDS),
  npp_image.select(RESPONSE)
]).float()

list = ee.List.repeat(1, KERNEL_SIZE)
lists = ee.List.repeat(list, KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)

arrays = featureStack.neighborhoodToArray(kernel)